## Script to delete round-robined data across FileDB

In [1]:
dest_folder_name = "sabl2048a"

In [2]:
import write_tools

In [3]:
%cd /home/idies/workspace/turb/

/home/idies/workspace/turb


In [4]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=write_tools.list_fileDB_folders()
folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/',
 '/home/idies/workspace/turb/data02_01/zarr/',
 '/home/idies/workspace/turb/data03_01/zarr/',
 '/home/idies/workspace/turb/data04_01/zarr/',
 '/home/idies/workspace/turb/data05_01/zarr/']

In [5]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/") # This is already created

In [6]:
# for i in range(len(folders)):
#     folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_prod/"

# folders[:5]

In [7]:
# get Z-order coordinates - they're part of the filename

cube_root = 4

z_order = []
# Z-order the cube of points so they "linearize" far from each other
for i in range(cube_root):
    a = []
    for j in range(cube_root):
        b = []
        for k in range(cube_root):
            z_position = write_tools.morton_pack(cube_root, i,j,k)
            b.append(z_position)
        a.append(b)
    z_order.append(a)

z_order

[[[0, 4, 32, 36], [2, 6, 34, 38], [16, 20, 48, 52], [18, 22, 50, 54]],
 [[1, 5, 33, 37], [3, 7, 35, 39], [17, 21, 49, 53], [19, 23, 51, 55]],
 [[8, 12, 40, 44], [10, 14, 42, 46], [24, 28, 56, 60], [26, 30, 58, 62]],
 [[9, 13, 41, 45], [11, 15, 43, 47], [25, 29, 57, 61], [27, 31, 59, 63]]]

In [8]:
# Calculate the number of chunks along each dimension
num_chunks = 4

# I want this to be a 3D list of lists
outer_dim = []

for i in range(num_chunks):
    mid_dim = []
    for j in range(num_chunks):
        inner_dim = []

        for k in range(num_chunks):
            inner_dim.append(str(i)+str(j)+str(k))

        mid_dim.append(inner_dim)

    outer_dim.append(mid_dim)

# outer_dim


In [9]:
folders_to_delete = []

for i in range(num_chunks):
    for j in range(num_chunks):
        for k in range(num_chunks):
            # Put e.g. z-order=4 to filedb05_01
            filedb_index = z_order[i][j][k] % len(folders) # Loop around
            current_array = outer_dim[i][j][k]
            
            folders_to_delete.append(folders[filedb_index] + "ariel_test_" + str(i) + str(j) + str(k) + ".zarr")
#             print(current_array)

folders_to_delete[:5]

['/home/idies/workspace/turb/data01_01/zarr/ariel_test_000.zarr',
 '/home/idies/workspace/turb/data05_01/zarr/ariel_test_001.zarr',
 '/home/idies/workspace/turb/data11_03/zarr/ariel_test_002.zarr',
 '/home/idies/workspace/turb/data03_01/zarr/ariel_test_003.zarr',
 '/home/idies/workspace/turb/data03_01/zarr/ariel_test_010.zarr']

In [10]:
import os
import concurrent.futures

In [11]:
# ChatGPT
def delete_folder(folder):
    try:
        os.system(f'rm -rf {folder}')
#         print(f"Deleted folder: {folder}")
    except Exception as e:
        print(f"Error deleting folder: {folder}")
        print(e)

In [12]:
# Create a ThreadPoolExecutor with the desired number of worker threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the delete_folder function for each folder to the executor
    future_to_folder = {executor.submit(delete_folder, folder): folder for folder in folders_to_delete}

    # Wait for the deletion tasks to complete
    for future in concurrent.futures.as_completed(future_to_folder):
        folder = future_to_folder[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error deleting folder: {folder}")
            print(e)